<a href="https://colab.research.google.com/github/MatheusLCouto/TAIA/blob/main/Ferramenta_de_Download_de_artigos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# --- Instala a biblioteca tqdm para a barra de progresso ---
!pip install tqdm -q

import os
import requests
from bs4 import BeautifulSoup
import re
import time
from google.colab import drive
from tqdm.notebook import tqdm # Importa a versão do tqdm otimizada para notebooks

# --- 1. Montar o Google Drive ---
# Ao executar esta célula, você precisará autorizar o acesso.
print("Montando o Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montado com sucesso!")
except Exception as e:
    print(f"Erro ao montar o Google Drive: {e}")
    # Se a montagem falhar, o script não pode continuar.
    exit()


def sanitize_filename(filename):
    """
    Remove caracteres inválidos de um nome de arquivo para que possa ser salvo
    no sistema de arquivos.
    """
    # Remove caracteres que não são letras, números, espaços, hífens ou pontos.
    sanitized = re.sub(r'[^\w\s\-\.]', '', filename.strip())
    # Substitui múltiplos espaços ou hífens por um único sublinhado.
    sanitized = re.sub(r'[-\s]+', '_', sanitized)
    return sanitized

def download_sbc_articles(url):
    """
    Baixa todos os artigos em PDF de uma página de anais da SBC SOL.
    Identifica a sigla do evento, cria a pasta correspondente e mostra o progresso.
    """

    # --- 2. Identificar a sigla do evento e definir a pasta de destino ---
    event_acronym = "EVENTO_DESCONHECIDO"
    match = re.search(r'index.php/(\w+)/', url)
    if match:
        event_acronym = match.group(1).upper()
        print(f"Sigla do evento identificada: {event_acronym}")
    else:
        print("Não foi possível identificar a sigla do evento na URL. Usando pasta padrão.")

    output_folder = f"/content/drive/MyDrive/leitura/{event_acronym}"

    # Verifica se a pasta já existe ou a cria
    if os.path.exists(output_folder):
        print(f"A pasta '{output_folder}' já existe. Os arquivos serão salvos nela.")
    else:
        print(f"Criando a estrutura de pastas '{output_folder}' no seu Google Drive...")
        os.makedirs(output_folder, exist_ok=True)


    print(f"\nAcessando a página principal dos anais: {url}")

    try:
        # Cabeçalhos para simular um navegador real
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        main_page_response = requests.get(url, headers=headers)
        main_page_response.raise_for_status() # Lança erro se a requisição falhar

        soup = BeautifulSoup(main_page_response.content, 'html.parser')

        article_summaries = soup.find_all('div', class_='obj_article_summary')

        if not article_summaries:
            print("Nenhum artigo encontrado na página. Verifique a URL ou a estrutura do site.")
            return

        print(f"Encontrados {len(article_summaries)} artigos. Iniciando o processo de download...")

        # Itera sobre os artigos com uma barra de progresso (tqdm)
        for index, summary in enumerate(tqdm(article_summaries, desc="Baixando artigos")):
            title_tag = summary.find('div', class_='title')
            title = title_tag.get_text(strip=True) if title_tag and title_tag.get_text(strip=True) else f"artigo_sem_titulo_{index}"

            reader_link_tag = summary.select_one('a.obj_galley_link.pdf')

            if not (reader_link_tag and reader_link_tag.has_attr('href')):
                # Não imprime aviso para não poluir a barra de progresso, mas pode ser reativado se necessário
                continue

            reader_url = reader_link_tag['href']

            if '/view/' not in reader_url:
                continue

            download_url = reader_url.replace('/view/', '/download/')

            try:
                # Gerar nome do arquivo e caminho final
                filename = sanitize_filename(title) + '.pdf'
                filepath = os.path.join(output_folder, filename)

                # Baixar o arquivo PDF diretamente
                pdf_response = requests.get(download_url, headers=headers)
                pdf_response.raise_for_status()

                with open(filepath, 'wb') as f:
                    f.write(pdf_response.content)

            except requests.exceptions.RequestException:
                # Se houver erro, apenas continua para o próximo, evitando parar o script.
                continue

            # Pausa para não sobrecarregar o servidor
            time.sleep(0.5)

        print(f"\nDownload de todos os artigos disponíveis concluído! Verifique a pasta '{output_folder}'.")

    except requests.exceptions.RequestException as e:
        print(f"ERRO CRÍTICO: Falha ao acessar a página principal. Motivo: {e}")

# --- Início da Execução ---
if __name__ == "__main__":
    # URL dos anais do XIX Simpósio Brasileiro de Sistemas de Informação (SBSI 2023)
    # Você pode trocar esta URL pela de outro evento do portal SOL da SBC.
    proceedings_url = "https://sol.sbc.org.br/index.php/kdmile/index"
    download_sbc_articles(proceedings_url)


Montando o Google Drive...
Mounted at /content/drive
Google Drive montado com sucesso!
Sigla do evento identificada: KDMILE
Criando a estrutura de pastas '/content/drive/MyDrive/leitura/KDMILE' no seu Google Drive...

Acessando a página principal dos anais: https://sol.sbc.org.br/index.php/kdmile/index
Encontrados 21 artigos. Iniciando o processo de download...


Baixando artigos:   0%|          | 0/21 [00:00<?, ?it/s]


Download de todos os artigos disponíveis concluído! Verifique a pasta '/content/drive/MyDrive/leitura/KDMILE'.
